In [2]:
import openmeteo_requests
import requests_cache
import pandas as pd
from retry_requests import retry

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after=3600)
retry_session = retry(cache_session, retries=5, backoff_factor=0.2)
openmeteo = openmeteo_requests.Client(session=retry_session)

# Arnhem Koordinaten
latitude = 51.9851
longitude = 5.8987

# Nur ein Tag zur Abfrage
url = "https://historical-forecast-api.open-meteo.com/v1/forecast"
params = {
    "latitude": latitude,
    "longitude": longitude,
    "start_date": "2024-08-17",
    "end_date": "2024-08-17",
    "hourly": "temperature_2m,precipitation,windspeed_10m"
}

# Request
responses = openmeteo.weather_api(url, params=params)
response = responses[0]

# Infos zum Standort
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()}{response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

# Hole stündliche Temperaturwerte
hourly = response.Hourly()
hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()
hourly_precipitation = hourly.Variables(1).ValuesAsNumpy()
hourly_windspeed_10m = hourly.Variables(2).ValuesAsNumpy()

hourly_data = pd.DataFrame({
    "date": pd.date_range(
        start=pd.to_datetime(hourly.Time(), unit="s", utc=True),
        end=pd.to_datetime(hourly.TimeEnd(), unit="s", utc=True),
        freq=pd.Timedelta(seconds=hourly.Interval()),
        inclusive="left"
    ),
    "temperature_2m": hourly_temperature_2m,
    "precipitation": hourly_precipitation,
    "windspeed_10m": hourly_windspeed_10m
})

# Konvertiere Zeit nach lokalem Arnhem-Zeit (optional)
hourly_data["date_local"] = hourly_data["date"].dt.tz_convert(response.Timezone())

# Filter auf genau 15:00 Uhr lokaler Zeit
target_row = hourly_data[hourly_data["date_local"].dt.strftime('%H:%M:%S') == "15:00:00"]

print("Wetter um 15:00 Uhr in Arnhem am 17.08.2024:")
print(target_row)


Coordinates 51.987998962402344°N 5.886999607086182°E
Elevation 37.0 m asl
Timezone NoneNone
Timezone difference to GMT+0 0 s
Wetter um 15:00 Uhr in Arnhem am 17.08.2024:
                        date  temperature_2m  precipitation  windspeed_10m  \
15 2024-08-17 15:00:00+00:00          24.152            0.0           5.76   

            date_local  
15 2024-08-17 15:00:00  


In [3]:
hourly_data

,date,temperature_2m,precipitation,windspeed_10m,date_local
0,2024-08-17 00:00:00+00:00,17.252001,0.3,6.840000,2024-08-17 00:00:00
1,2024-08-17 01:00:00+00:00,18.402000,0.8,5.040000,2024-08-17 01:00:00
2,2024-08-17 02:00:00+00:00,17.502001,0.0,7.559999,2024-08-17 02:00:00
3,2024-08-17 03:00:00+00:00,16.302000,0.0,8.640000,2024-08-17 03:00:00
4,2024-08-17 04:00:00+00:00,17.152000,0.3,10.080000,2024-08-17 04:00:00
5,2024-08-17 05:00:00+00:00,16.552000,0.0,9.720000,2024-08-17 05:00:00
6,2024-08-17 06:00:00+00:00,17.252001,0.0,6.840000,2024-08-17 06:00:00
7,2024-08-17 07:00:00+00:00,18.101999,0.0,9.360000,2024-08-17 07:00:00
8,2024-08-17 08:00:00+00:00,18.802000,0.0,6.480000,2024-08-17 08:00:00
9,2024-08-17 09:00:00+00:00,20.252001,0.0,4.680000,2024-08-17 09:00:00
